# Preparació de classificador binari

En aquest document es mostra la preparació realitzada del classificador binari, també encarregat de generar les explicacions contrafactuals.

In [39]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from pandas.api.types import is_string_dtype
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from aix360.algorithms.nncontrastive import NearestNeighborContrastiveExplainer
from sklearn.tree import DecisionTreeClassifier
from joblib import dump, load

In [40]:
# Nom del fitxer on s'emmagatzemarà el fitxer amb el model entrenat
saved_model_file = "decision_tree_model.joblib"
saved_explainer_model = "explainer_model.joblib"

Per a aquesta primera part, prepararem el conjunt de dades. Primer haurem de carregar les dades del fitxer csv. després realitzarem un tractament de les dades. Primer de tot, donat que la llibreria no accepta valors Nan, substituirem els valors Nan per un nou valor "unknown". 

Finalment, codificarem les dades categoriques i normalitzarem aquelles dades númeriques, donat que el model no accepta les variables catègoriques en el seu format original (string). Per fer-ho, es fara servir una codificació one-hot encode, en el que es crea una nova columna per a cada possible valor de cada variable categòrica i en cas de tenir aquesta variable, s'indica com 1 i en cas contrari com un 0, generant un vector binari.

Per als valors númerics, es calcularà de cada columna el valor màxim i minim, i per calcular el nou valor es farà servir la següent formula:

$$
Valor = \frac{valor - valorMínim}{valorMàxim - valorMínim}
$$

In [41]:
data = pd.read_csv('./archive/german_credit_data.csv')
data_copy = data.copy()
# data_copy = data_copy.drop("Id", axis=1)
data_copy.fillna(value="unknown", inplace=True)
labels = data_copy.columns

dict_min_max_by_col = {}

def normalize(df, col_name):
    result = df.copy()

    # Calculem valors minims i maxims de cada columa i els emmagatzem a un diccionari
    max_value = df.max()
    min_value = df.min()
    dict_min_max_by_col[col] = (min_value, max_value)

    # Normalitzem amb la formula: normalized = ((value - min_value) / (max_value - min_value))
    result = ((df - min_value) / (max_value - min_value))
    return result

for col in labels:
    # En cas que la columna sigui categòrica
    if is_string_dtype(data_copy[col]) or col == "Job":
        if col == 'Risk':
            # En cas que sigui la columna Risk, donarem valor 1 si es good i 0 si és false
            values, uniques = pd.factorize(data_copy[col]) #PD Factorize da un valor numerico para cada posible valor de la columna
            # por lo que al encontrar primero un good, good = 0 y bad = 1  
            data_copy[col] = data_copy[col].apply(lambda x: 1 if x == 'good' else 0)
            continue

        # Per a la resta de columnes, es crearà una columna per cada valor de cada atribut i s'omplirà de forma binaria
        data_copy = pd.concat([data_copy, pd.get_dummies(data_copy[col], prefix=col)], axis=1)
        data_copy.drop(col, axis=1, inplace=True)
    else:
        # Normalitzem els valors númerics de la columna 
        data_copy[col] = normalize(data_copy[col], col)


# Afegim al final del dataset la columna risk
data_copy = data_copy[[c for c in data_copy if c not in ["Risk"]] + ["Risk"]]
display(data_copy)

# Variables necesarias:
columns_data = data_copy.columns
print(data_copy.columns.size)

,Age,Credit amount,Duration,Sex_female,Sex_male,Job_0,Job_1,Job_2,Job_3,Housing_free,...,Checking account_unknown,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Risk
0,0.857143,0.050567,0.029412,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
1,0.053571,0.313690,0.647059,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0.535714,0.101574,0.117647,0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
3,0.464286,0.419941,0.558824,0,1,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,1
4,0.607143,0.254209,0.294118,0,1,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.214286,0.081765,0.117647,1,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,1
996,0.375000,0.198470,0.382353,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
997,0.339286,0.030483,0.117647,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,1
998,0.071429,0.087763,0.602941,0,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0


30


Un cop finalitzat el tractament, haurem de crear els conjunts de dades per a entrenament i test. Per fer-ho, haurem de separar les dades d'entrada i dades de sortida (X i Y respectivament).

- Les dades d'entrada són aquells atributs els quals no volem predir, sinó que són aquells que ens ajuden a donar la predicció.
- Les dades de sortida són aquelles dades qeu retornarem en la predicció del model.

Per fer-ho, usarem la funció train_test_split, la qual ens permetra realitzar la separació dels sets d'entrada d'entrenament i test i els sets de sortida d'entrenament i test. Usarem un tamany de 25% de les dades totals per el test, i el restant per l'entrenament.

In [42]:
# Separem les dades per entrada (X) i sortida (y)
X = data_copy.drop("Risk", axis=1) # Dades d'entrada
Y = data_copy['Risk'] # Dades de sortida

# Separem el dataset en conjunts de test i entrenament
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

Un cop preparades les dades, comencarem a entrenar el model. En aquest cas, per evitar entrenar els models més d'un cop sense necessitat, salvarem l'entrenament en un fitxer, i en cas que aquest fitxer existeixi, no es tornarà a entrenar el model, sinó que es carregarà el contingut del fitxer en el model. En cas que el fitxer no existeixi, es crearà el model de 0 i s'entrenarà el model amb les dades d'entrenament. Posteriorment, es guardarà en el fitxer que es cercarà en el condicional, per evitar que el model es torni a entrenar.

Després, calcularem un accuracy tant per al train com per al test.

In [43]:
# Comprovem si ja hi ha un model entrenat
if not os.path.exists(saved_model_file):
    model = DecisionTreeClassifier(
        max_depth=10,
        max_leaf_nodes=10
    )
   
    model.fit(x_train.values, y_train.values)

    # Guardem el model entrenat en el fitxer saved_model_file
    dump(model, saved_model_file)
    print("Model guardat en el fitxer", saved_model_file)

else:
    # Carreguem aquell model que ja es trobés preparat.
    print("Cargant model ja existent...")
    model = load(saved_model_file)

# Traiem valors de precissió
scores = cross_val_score(model, x_train, y_train, cv=5)
print("Puntuació de l'entrenament:", scores.mean())

scores = cross_val_score(model, x_test, y_test, cv=5)
print("Puntuació del test:", scores.mean())

Cargant model ja existent...
Puntuació de l'entrenament: 0.704
Puntuació del test: 0.6839999999999999


En aquest moment, ja tenim preparada la primera part del classificador, el model que s'encarregarà de generar les prediccions dels crèdits. El següent pas serà preparar l'algorisme encarregat de generar les explicacions contrafactuals.

L'algorisme que s'ha seleccionat per a aquest projecte és **Nearest Neighbor Contrastive Explainer**, un algorisme que genera veins el més proper possible de la instància pasada com a paràmetre, però amb una classificació diferent. En aquest cas, donat que el classificador és un classificador binari, en cas de passar-li una instància de sol·licitud de crèdit classificada com a denegat(Bad), els veins que generi es classificaran com a concedit (Good).

In [44]:
# Creem el explicador i l'entrenem per a que pugui generar l'explicacions (si no s'entrena, retorna error la funció de explain_instance)
explainer = NearestNeighborContrastiveExplainer(model=model.predict, n_classes=2, neighbors=10)
explainer.fit(X, epochs=50, batch_size=100)

Un cop tenim ja preparat el mètode explicatiu, podem començar a generar explicacions contrafactuals de sol·licituds classificades com denegades. Per fer un breu test, a la següent cel·la es prepara un subconjunt amb instàncies de credits classsificats com denegats (Bad) i una variable index, la qual ens permet escollir d'aquest conjunt de dades una instància.

In [45]:
negatives_cases = data_copy.loc[data_copy.Risk == 0] # Cerquem aquells casos negatius

# Index de la instància a provar
index = 0

# Preparem la instància negativa
ins = negatives_cases.iloc[index: index + 1]
ins = ins.drop("Risk", axis=1)

nearest_benign_constrastive = explainer.explain_instance(ins)

26/26 [==============================] - 0s 991us/step


In [46]:
# Funció que permet reescalar el valor per a poder mostrar a la taula els valors de la sol·licitud sense processar
def rescale(row, col):
    row_copy = row.copy()
    if col in dict_min_max_by_col:
        min_value, max_value = dict_min_max_by_col[col]
        row_copy = (np.multiply(row, (max_value - min_value)) + min_value)
    return row_copy

def rescale_prediction(row, columns, top=3):
    for i in range(top):
        column = columns[i]
        min_value, max_value = dict_min_max_by_col[column]
        row[i] = (np.multiply(row[i], (max_value - min_value)) + min_value)
    
    return row

En aquest moment, l'explicador ja ha generat un veí semblant a la sol·licitud de crèdit original amb una classificació diferent, o sigui, classificat com concedit.

In [47]:
original_value = ins.copy() # X
original_value['Risk'] = 0.0

for i in original_value.columns:
    original_value[i] = rescale(original_value[i].values, i)

predicted_value = np.array(nearest_benign_constrastive[0]["neighbors"][0])
predicted_value_with_risk = np.append(predicted_value, 
                                      model.predict(predicted_value.reshape(1, predicted_value.shape[0])))

predicted_value_with_risk = rescale_prediction(predicted_value_with_risk, original_value.columns)
predicted_value_with_risk = predicted_value_with_risk.reshape(1, predicted_value_with_risk.shape[0])

delta = predicted_value_with_risk - original_value
X3 = np.vstack((original_value, predicted_value_with_risk, delta))
dfre = pd.DataFrame.from_records(X3)
dfre.columns = original_value.columns

dfre.rename(index={0: 'Original value', 1: 'Predicted value', 2: "Difference between instances"}, inplace=True)

for i in range(len(dfre['Risk']) - 1):
    if dfre["Risk"][i] == 0:
        dfre["Risk"][i] = "Bad"
    else:
        dfre["Risk"][i] = "Good" 

dfret = dfre.transpose()

def highlight_ce(s, col, ncols):
    if (type(s[col]) != str):
        if (s[col] != 0):
            return(["background-color: yellow"] * ncols)
    return (["background-color: white"] * ncols)

dfret.style.apply(highlight_ce, col="Difference between instances", ncols=3, axis=1)

,Original value,Predicted value,Difference between instances
Age,22.000000,20.000000,-2.000000
Credit amount,5951.000000,1967.000000,-3984.000000
Duration,48.000000,24.000000,-24.000000
Sex_female,1.000000,1.000000,0.000000
Sex_male,0.000000,0.000000,0.000000
Job_0,0.000000,0.000000,0.000000
Job_1,0.000000,0.000000,0.000000
Job_2,1.000000,1.000000,0.000000
Job_3,0.000000,0.000000,0.000000
Housing_free,0.000000,0.000000,0.000000


La taula que es mostra a la cel·la superior mostra la següent informació:

- Primera columna: les característiques de una sol·lciitud de crèdit
- Segona columna: sol·licitud de crèdit original
- Tercera columna: Veí generat amb classificació diferent
- Quarta columna: diferencies entre el la sol·licitud inicial i la generada

A més, aquells atributs que han canviat, apareixen marcats en color groc. A les cel·les posteriors, mostrem una implementació que ens permet generar una taula que mostra els tres veins més propers generats.

In [48]:
# Aquesta funció s'encarrega de generar la taula rebent com a paràmetre la sol·licitud original i els veins generats per l'explicador
def show_counterfactual_table(original_data, nb_contrastive):
    original_value = original_data.copy()
    num_neighbors = len(nb_contrastive[0]["neighbors"])
    index = {0:"Original input"}
    predictions = []
    final_result = ["Bad"]
    columns = original_value.columns
    ind_val = 0
    num_iterations = 3

    # Invertim els valors de la normalització dels valors
    for col in columns:
        original_value[col] = rescale(original_value[col].values, col)

    if num_neighbors < num_iterations:
        num_iterations = num_neighbors
    
    for i in range(num_iterations):
        #Calculem la predicció per a cada vei generat
        predicted_value = np.array(nb_contrastive[0]["neighbors"][i])
        predicted_value_with_risk = np.append(predicted_value, model.predict(predicted_value.reshape(1, predicted_value.shape[0])))
        predicted_value_with_risk = rescale_prediction(predicted_value_with_risk, original_value.columns)
        predicted_value_with_risk = predicted_value_with_risk.reshape(1, predicted_value_with_risk.shape[0])

        # Si la predicció del vei retorna 1, es guarda un good a la llista de resultat, en cas contrari un Bad
        if int(predicted_value_with_risk[0][-1]) == 1:
            final_result.append("Good")
        else:
            final_result.append("Bad")

        # Afegim un NIL per a les diferencies
        final_result.append("NIL")

        # Ajustem les capçaleres de la taula
        index[ind_val + 1] = "Counterfactual " + str(i + 1)
        index[ind_val + 2] = "Difference"
        ind_val = ind_val + 2
        
        predictions.append(predicted_value_with_risk)

    # Ajuntem les columnes per mostrar-ho en un dataframe
    v_stack = create_vstacks(original_value, predictions)
    dfre = pd.DataFrame.from_records(v_stack)
    dfre.columns = original_value.columns

    dfre.rename(index=index, inplace=True)
    len_ = len(dfre["Risk"])
    dfre["Risk"][0] = "Bad"

    # La ultima cel·la del dataframe afegirem la classificació que s'ha calculat anteriorment
    for i in range(1, len_):
        dfre["Risk"][i] = final_result[i]

    return dfre.transpose()

# Aquesta funció s'encarrega d'ajuntar els columnes de la sol·licitud original, el vei generat i les diferencies
def create_vstacks(original_data, predictions):
    values = (original_data, )
    for prediction in predictions:
        delta = prediction - original_data
        values = values + (prediction, )
        values = values + (delta, )

    return np.vstack(values)


# Aquesta funció s'encarrega de preparar la sol·licitud rebuda com a parametre amb el format per al model
def prepare_instance(df: pd.DataFrame):
    list_clients = []

    for col, row in df.iterrows():
        prepared_client = {
            "Age": 0,
            "Credit amount": 0,
            "Duration": 0,
            "Sex_female": 0,
            "Sex_male": 0,
            "Job_0": 0,
            "Job_1": 0,
            "Job_2": 0,
            "Job_3": 0,
            "Housing_free": 0,
            "Housing_rent": 0,
            "Housing_own": 0,
            "Saving accounts_little": 0,
            'Saving accounts_moderate': 0,
            'Saving accounts_quite rich': 0,
            'Saving accounts_rich': 0,
            'Saving accounts_unknown': 0,
            'Checking account_little': 0,
            'Checking account_moderate': 0,
            'Checking account_rich': 0,
            'Checking account_unknown': 0,
            'Purpose_business': 0,
            'Purpose_car': 0,
            'Purpose_domestic appliances': 0,
            'Purpose_education': 0,
            'Purpose_furniture/equipment': 0,
            'Purpose_radio/TV': 0,
            'Purpose_repairs': 0,
            'Purpose_vacation/others': 0,
        }

        # extraiem de la row totes les característiques del crèdit
        age, sex, job, housing, savingAccount, checkingAccount, creditAmount, duration, purpose = row
        
        # Normalitzem els valors numerics
        new_age = normalizeValue(age, dict_min_max_by_col['Age'])
        new_amount = normalizeValue(creditAmount, dict_min_max_by_col['Credit amount'])
        new_duration = normalizeValue(duration, dict_min_max_by_col['Duration'])

        prepared_client["Age"] = new_age
        prepared_client["Credit amount"] = new_amount
        prepared_client["Duration"] = new_duration

        # Assignem el valor 1 per als valors categorics presents a la sol·licitud
        prepared_client["Sex_" + sex] = 1
        prepared_client["Job_" + job] = 1
        prepared_client["Housing_" + housing] = 1
        prepared_client["Saving accounts_" + savingAccount] = 1
        prepared_client["Checking account_" + checkingAccount] = 1
        prepared_client["Purpose_" + purpose] = 1

        list_clients.append(prepared_client)

    # Creem un dataframe amb les dades de les sol·lciituds 
    result = pd.DataFrame(list_clients, columns=columns_data)
    result = result.drop('Risk', axis=1)
    return result

# Funció de normalització dels valors númerics introduits a la variable
def normalizeValue(value: int, min_max: tuple):
    return ((value - min_max[0]) / (min_max[1] - min_max[0]))
        

Per provar que el sistema funciona, es planteja la següent cel·la. Tenim les següents variables:

- loan: consisteix en la instància del crèdit.
- Columns: consisteix en les columnes que formen el dataset. Aquesta variable no s'ha de tocar per evitar errors.

Al introduir una sol·licitud de crèdit, generarà primer un dataframe amb els valors i realtizarà una predicció de la classificació del crèdit. En cas que la classificació resulti en denegat, retornarà una taula amb tres contrafactuals, que realitza els canvis necessaris per a que la classificació del crèdit canviï a concedit. En cas contrari, únicament mostrarà un missatge confirmant que el crèdit s'ha classificat com a concedit.

In [49]:
# loan es pot canviar per a qualsevol exemple crèdit.
loan = [[60, "male", "3", "own", "unknown", "moderate", 40000, 25, "car"]]
columns = ["Age", "Sex", "Job", "Housing", "Saving accounts", "Checking account", "Credit amount", "Duration", "Purpose"]

instance = pd.DataFrame(loan, columns=columns)
instance_prepared = prepare_instance(instance)
_class = model.predict(instance_prepared.values)

# Comprovem la classificaciO del credit [1] = Concedit, [0] = Denegat
if np.array_equal(_class, np.ndarray([1])):
    print("El crèdit introduït s'ha classificat com denegat\nGenerant altres opcions...")
    new_explain = explainer.explain_instance(instance_prepared, neighbors=10)
    instance_prepared['Risk'] = _class
    dataframe = show_counterfactual_table(instance_prepared, new_explain)
    display(dataframe)

else:
    print("El crèdit introduït s'ha classificat com concedit")

El crèdit introduït s'ha classificat com denegat
Generant altres opcions...
26/26 [==============================] - 0s 1ms/step


,Original input,Counterfactual 1,Difference,Counterfactual 2,Difference,Counterfactual 3,Difference
Age,60.0,29.0,-31.0,60.0,0.0,49.0,-11.0
Credit amount,40000.0,2333.0,-37667.0,6468.0,-33532.0,1262.0,-38738.0
Duration,25.0,24.0,-1.0,20.0,-5.0,12.0,-13.0
Sex_female,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sex_male,1.0,1.0,0.0,1.0,0.0,1.0,0.0
Job_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Job_1,0.0,1.0,1.0,0.0,0.0,1.0,1.0
Job_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Job_3,1.0,0.0,-1.0,1.0,0.0,0.0,-1.0
Housing_free,0.0,0.0,0.0,0.0,0.0,0.0,0.0
